In [1]:
!pip install wandb
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [2]:
!pip install ultralytics
from IPython import display
display.clear_output()

import ultralytics

In [3]:
from ultralytics import YOLO

from IPython.display import display, Image

In [4]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="lqigRvMBwxOG5h9OGHlq")
project = rf.workspace("xxxxx").project("yellow-ilijm")
version = project.version(4)
dataset = version.download("yolov8")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.9/74.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 30.7 MB/s eta 0:00:0000:01:00:01
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycler 0.12.1
    Uninstalling cycler-0.12.1:
      Successfully uninstalled cycler-0.12.1
  Attempting uninstall: certifi
    Found existing installation: certifi 2


Extracting Dataset Version Zip to yellow-4 in yolov8:: 100%|██████████| 1312/1312 [00:00<00:00, 8586.42it/s]


In [5]:
%%writefile /kaggle/working/yellow-4/data.yaml
names:
- yellow smoke
nc: 1
roboflow:
  license: MIT
  project: yellow-ilijm
  url: https://universe.roboflow.com/xxxxx/yellow-ilijm/dataset/4
  version: 4
  workspace: xxxxx
test: ../test/images
train: ../train/images
val: ../valid/images

Overwriting /kaggle/working/yellow-4/data.yaml


In [21]:
sweep_config = {
    'method': 'random' 
    }
print("Finished Setup")

Finished Setup


In [22]:
metric = {
    'name': 'metrics/mAP50-95(B)',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric
print("Finished Setup")

Finished Setup


In [29]:
parameters_dict = {
    'epochs': {
        'values': [5,15,25]
    },
    'batch': {
        'values': [16,32]
    },
    'lr0': {

        'distribution': 'uniform',
        'min': 0.0005,
        'max': 0.005
    },
    'lrf': {
        # a flat distribution
        'distribution': 'uniform',
        'min': 0.001,
        'max': 0.01
    },
}

sweep_config['parameters'] = parameters_dict
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'metrics/mAP50-95(B)'},
 'parameters': {'batch': {'values': [16, 32]},
                'epochs': {'values': [5, 15, 25]},
                'lr0': {'distribution': 'uniform', 'max': 0.005, 'min': 0.0005},
                'lrf': {'distribution': 'uniform', 'max': 0.01, 'min': 0.001}}}


In [30]:
sweep_id = wandb.sweep(sweep_config, project="hyper")

Create sweep with ID: y350t980
Sweep URL: https://wandb.ai/pirajesh1/hyper/sweeps/y350t980


In [10]:
import os

In [11]:
model = YOLO('yolov8s.pt')

100%|██████████| 21.5M/21.5M [00:00<00:00, 273MB/s]


In [31]:
def train(config=None):
    wandb.init(config=config, tags=["trial3"]) # Initialize a new W&B Run
    config = wandb.config # Config set by Sweep Controller
    
    # Train function
    results = model.train(data='/kaggle/working/yellow-4/data.yaml',
                          epochs = config.epochs,
                          batch = config.batch,
                          lr0 = config.lr0,
                          lrf = config.lrf,
                          project = "hyper", imgsz = 800, plots = True)


wandb.agent(sweep_id, train, count=10)

wandb: Agent Starting Run: vfa2n87f with config:
wandb: 	batch: 16
wandb: 	epochs: 5
wandb: 	lr0: 0.0008702496532952626
wandb: 	lrf: 0.003926182955791735


Ultralytics YOLOv8.2.18 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/yellow-4/data.yaml, epochs=5, time=None, patience=50, batch=16, imgsz=800, save=True, save_period=1, cache=False, device=None, workers=8, project=hyper, name=train, exist_ok=True, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

train: Scanning /kaggle/working/yellow-4/train/labels.cache... 600 images, 0 backgrounds, 0 corrupt: 100%|██████████| 600/600 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/yellow-4/valid/labels.cache... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<?, ?it/s]


...

Run ja0cyk1v errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/1789913246.py", line 6, in train
    results = model.train(data='/kaggle/working/yellow-4/data.yaml',
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/model.py", line 674, in train
    self.trainer.train()
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 199, in train
    self._do_train(world_size)
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 313, in _do_train
    self._setup_train(world_size)
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 228, in _setup_train
    self.model = self.model.to(self.device)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1160, in to
    return self._apply(convert)
  File "/opt/conda/lib/python3.10

Ultralytics YOLOv8.2.18 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/yellow-4/data.yaml, epochs=15, time=None, patience=50, batch=32, imgsz=800, save=True, save_period=1, cache=False, device=None, workers=8, project=hyper, name=train, exist_ok=True, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_con

Run 1afibxyv errored:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/wandb/agents/pyagent.py", line 308, in _run_job
    self._function()
  File "/tmp/ipykernel_34/1789913246.py", line 6, in train
    results = model.train(data='/kaggle/working/yellow-4/data.yaml',
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/model.py", line 674, in train
    self.trainer.train()
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 199, in train
    self._do_train(world_size)
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 313, in _do_train
    self._setup_train(world_size)
  File "/opt/conda/lib/python3.10/site-packages/ultralytics/engine/trainer.py", line 228, in _setup_train
    self.model = self.model.to(self.device)
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1160, in to
    return self._apply(convert)
  File "/opt/conda/lib/python3.10

Ultralytics YOLOv8.2.18 🚀 Python-3.10.13 torch-2.1.2 CUDA:0 (Tesla P100-PCIE-16GB, 16276MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/kaggle/working/yellow-4/data.yaml, epochs=5, time=None, patience=50, batch=32, imgsz=800, save=True, save_period=1, cache=False, device=None, workers=8, project=hyper, name=train, exist_ok=True, pretrained=True, optimizer=Adam, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=False, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf

wandb: Ctrl + C detected. Stopping sweep.
